In [1]:
import gzip
import os
import multiprocessing as mp
import pickle

In [2]:
data_name = 'Clothing_Shoes_and_Jewelry_5'
data_file = 'data/%s.json.gz' % data_name
batches_folder = 'data/%s' % data_name
num_threads = mp.cpu_count()
batch_size = 8192

In [3]:
def get_next_filename(prefix='batch', suffix='pkl'):
    number = 0
    while True:
        number += 1
        yield '%s_%06d.%s' % (prefix, number, suffix)

In [4]:
lines = []
name_gen = get_next_filename(batches_folder+'/batch', 'json')
os.makedirs(batches_folder, exist_ok=True) 

with open('data/Clothing_Shoes_and_Jewelry_5.json', 'rt') as file_read:
    for line in file_read:
        lines.append(line)
        if len(lines) == batch_size:
            filepath = next(name_gen)
            with open(filepath, 'wt') as file_write:
                file_write.writelines(lines)
            lines = []

In [5]:
batch_files = os.listdir(batches_folder)
batch_files = [ name for name in batch_files if '.json' in name ]
batch_files.sort()
batch_files = [ os.path.join(batches_folder, name) for name in batch_files ]
len(batch_files)

1377

In [6]:
def itemize(batch_file):
    with open(batch_file, 'r') as file_read:
        lines = file_read.readlines()
    items =  [ eval(line.replace('true', 'True').replace('false', 'False')) for line in lines ] 
    new_name = batch_file.split('.')[0]+'.pkl'
    with open(new_name, 'wb') as file_write:
        pickle.dump(items, file_write)
    return len(items)

In [7]:
with mp.Pool(num_threads) as p:
    items_batched_count = p.map(itemize, batch_files)

In [8]:
batch_files = os.listdir(batches_folder)
batch_files = [ name for name in batch_files if '.pkl' in name ]
batch_files.sort()
batch_files = [ os.path.join(batches_folder, name) for name in batch_files ]
len(batch_files)

1377

In [9]:
for batch_file in batch_files:
    with open(batch_file, 'rb') as file_read:
        lines = pickle.load(file_read)
    break

In [10]:
lines[0]

{'overall': 5.0,
 'vote': '2',
 'verified': True,
 'reviewTime': '05 4, 2014',
 'reviewerID': 'A2IC3NZN488KWK',
 'asin': '0871167042',
 'style': {'Format:': ' Paperback'},
 'reviewerName': 'Ruby Tulip',
 'reviewText': 'This book has beautiful photos, good and understandable directions, and many different kinds of jewelry.  Wire working and metalsmithing jewelry are covered.  Highly recommend this book.',
 'summary': 'Unique designs',
 'unixReviewTime': 1399161600}

In [41]:
all_files = os.listdir(batches_folder)
all_files = [ os.path.join(batches_folder, name) for name in all_files ]
json_files = [ name for name in all_files if '.json' in name ]
json_files.sort()
pkl_files = [ name for name in all_files if '.pkl' in name ]
pkl_files.sort()
print(len(json_files), len(pkl_files))
assert len(json_files) == len(pkl_files)

1377 1377


In [42]:
with open(pkl_files[-1], 'rb') as file_read:
    lines = pickle.load(file_read)
    final_line_pkl = lines[0]
print(final_line_pkl)

{'image': ['https://images-na.ssl-images-amazon.com/images/I/51rFo+LoQDL._SY88.jpg'], 'overall': 5.0, 'verified': True, 'reviewTime': '04 15, 2018', 'reviewerID': 'AIA1NXRDU69S4', 'asin': 'B01H0F99TA', 'style': {'Size:': ' 12 m', 'Color:': ' Bright Red'}, 'reviewerName': 'SherrySeattle', 'reviewText': 'This is one of the cutest outfits ever.  Im buying a second one for my daughter.  I can tell she loves it.  Very soft and stretchy and comfortable.  Shes a year old and the 12 month fits her great.', 'summary': 'A must have', 'unixReviewTime': 1523750400}


In [47]:
with open(json_files[-1], 'r') as file_read:
    lines = file_read.readlines()
    final_line_json = lines[0]
print(final_line_json)

{"image": ["https://images-na.ssl-images-amazon.com/images/I/51rFo+LoQDL._SY88.jpg"], "overall": 5.0, "verified": true, "reviewTime": "04 15, 2018", "reviewerID": "AIA1NXRDU69S4", "asin": "B01H0F99TA", "style": {"Size:": " 12 m", "Color:": " Bright Red"}, "reviewerName": "SherrySeattle", "reviewText": "This is one of the cutest outfits ever.  Im buying a second one for my daughter.  I can tell she loves it.  Very soft and stretchy and comfortable.  Shes a year old and the 12 month fits her great.", "summary": "A must have", "unixReviewTime": 1523750400}



In [48]:
final_line_json = eval(final_line_json.replace('true', 'True').replace('false', 'False'))
assert final_line_pkl == final_line_json